# Lab 01 (Đồ án 1) "Color Compression"

**Môn**: MTH00051 "Toán ứng dụng và thống kê" @ 18CLC4

**SV**: 18127221, Bùi Văn Thiện

---

Centroid chỉ được chọn ngẫu nhiên từ trong ảnh.

## Tham số đầu vào

In [ ]:
k = 3                            # số cluster / số màu đích
max_iterations = 1000            # chặn: số iteration tối đa quá trình k-means có thể chạy
convergence_threshold = 10e-3    # chặn: sai số khi so sánh bằng các centroid
filename = 'color_glass_rainbow' # tên file: giả định có đuôi jpg

## Import + hàm công cụ

In [ ]:
# Python built-ins
import copy
import os
import time

# Externals
import numpy as np
from PIL import Image
from matplotlib import pyplot

In [ ]:
np.set_printoptions(precision=2)    # numpy.ndarray.__str__: giới hạn lại 2 chữ số thập phân
tnow = time.perf_counter            # dùng để đo thời gian chạy một iteration
show = pyplot.imshow                # hiện ảnh (trên Notebook)
input_file = 'input/{f}.jpg'.format(f=filename)
output_file = 'output/{f}_{k}.jpg'.format(f=filename, k=k)

## Tiền xử lý

Mở ảnh và reshape

In [ ]:
image = np.array(Image.open(input_file), dtype=np.float64)
width, height, channels = image.shape
image = image.reshape((width*height, channels))

## Giảm màu

Chọn ngẫu nhiên $k$ centroid **không trùng nhau về mặt giá trị** làm điểm bắt đầu

In [ ]:
get_random_pixel = lambda: image[np.random.randint(0, image.shape[0])]
centroids = np.array([get_random_pixel()])
while len(centroids) < k:
    random_pixel = np.array([get_random_pixel()])
    check = (random_pixel == centroids).all(1).any()
    if not check:
        centroids = np.concatenate((centroids, random_pixel), axis=0)

print('{} centroid khởi điểm'.format(k))
print(' '.join(list(map(str, centroids))))

Bắt đầu vòng lặp: label từng điểm ảnh -- cập nhật centroid theo các trung bình các cụm

In [ ]:
for i in range(max_iterations):
    last_known_centroids = copy.deepcopy(centroids)
    
    t = tnow()
    
    # Đánh label cho từng điểm ảnh
    # Tham khảo từ Real Python: https://realpython.com/numpy-array-programming/#clustering-algorithms
    labels = np.argmin(np.linalg.norm(image - centroids[:, None], axis=2), axis=0)
    
    for label in range(k):
        # Lọc từng điểm thuộc label
        point_labels = image[labels == label]
        # Tính trung bình trên tập các điểm đó
        centroids[label] = np.mean(point_labels, axis=0)
    
    t = tnow() - t
    print('{i1}/{i2}; {t:.2f}s; {c}'.format(
        i1 = i+1,
        i2 = max_iterations,
        t = t,
        c = ' '.join(list(map(str, centroids)))
    ))
    
    if np.allclose(last_known_centroids, centroids, rtol=convergence_threshold, equal_nan=False):
        break

## Hậu xử lý + hiện/lưu ảnh

Thực hiện các công việc:
- Làm tròn các giá trị trong centroid (để map trực tiếp qua mã màu RGB)
- Dựng lại ảnh (map từng giá trị trong label với centroid tương ứng, sau đó reshape lại thành ảnh có kích thước gốc)
- Hiện ảnh (bằng MatPlotLib) + ghi ảnh ra file

In [ ]:
c_rounded_off = [[int(e) for e in each_centroid] for each_centroid in centroids]

new_image = np.array([c_rounded_off[int(each_pixel)] for each_pixel in labels])
new_image = new_image.reshape((width, height, channels))
show(new_image)

new_image_file = Image.fromarray(new_image.astype(np.uint8))
new_image_file.save(output_file)